<a href="https://colab.research.google.com/github/salarbalou/photothermal_heating/blob/main/Photothermal_Study_of_CQDs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import plotly.io as pio
pio.templates.default= 'plotly_dark'
from plotly.subplots import make_subplots

In [47]:
!wget 'https://raw.githubusercontent.com/salarbalou/steadystate_analysis/main/10-21-21-CQD-10x-dc90.txt'

--2021-11-23 02:54:58--  https://raw.githubusercontent.com/salarbalou/steadystate_analysis/main/10-21-21-CQD-10x-dc90.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.111.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4677 (4.6K) [text/plain]
Saving to: ‘10-21-21-CQD-10x-dc90.txt’

10-21-21-CQD-10x-dc 100%[===================>]   4.57K  --.-KB/s    in 0s      

2021-11-23 02:54:58 (68.4 MB/s) - ‘10-21-21-CQD-10x-dc90.txt’ saved [4677/4677]



In [76]:
 data = np.loadtxt('https://raw.githubusercontent.com/salarbalou/steadystate_analysis/main/10-21-21-CQD-10x-dc90.txt', delimiter=' , ')
 data = data[23:]

In [78]:
#for blue 435nm maxpower=4W laser:
m, b = 7.025524475524473, -66.07575757575763
# dc is arduino board pwm pin duty cycle
dc = np.linspace(10, 150, 15) # at 150 dc we will reach 1W of power for laser

In [132]:
# *** Steady State Analysis ***
t = data[:, 0] #time (s)
T = data[:, 1] #temperature of the solution (C)
Tw = np.full(t.size, 22)#temperature of the environment (C) (needs to be measured by another K type thermocouple) assumed to be constant at 22 C
R  = np.gradient(T, t) #gradient of T with respect to time
HR = np.max(R) #heating rate
Teq = T[np.min(np.where(R<0.01))] #equilibrium temperature (C)
teq = t[np.min(np.where(R<0.01))] # time required to reach equilibrium (s)
print(f'Reached equilibrium eemperature of {Teq} in {teq} seconds')

Reached equilibrium eemperature of 94.14 in 81.0 seconds


In [ ]:
#Iout = #measured using powermeter: f(CCQD, L_laser) (W)
#S_PCR_tube = #surface area of the solution at which laser beam is passing through:[PCR_tube, paper] (m**2)
#S_paper = #surface are of the paper at which laser beam is covering:[PCR_tube, paper]  (m**2)
#m = rho * V #mass of solution (Kg)

In [80]:
Iin = m*dc + b #input power of the laser: f(dc) (W)
h = 1000 #heat transfor coefficient of solution with its surrounding (W/(m**2)*C)

rho  = 1000 #density of solution (kg/m**3)
V = 1e-8 #volume of the solution (m**3)
Cp = 4182 #heat capacity of water (J/Kg*C)

In [120]:
#C = der(T, t[60:]) #cooling rate (C/s) #turning off laser for 60 seconds to measure the cooling rate

In [126]:
HR = np.max(R)

In [133]:
nx = 100
nt = t.size #temperature rise is evaluated in 60 seconds

In [9]:
fig = make_subplots()
fig.add_scatter(x = np.arange(nx), y = np.full(nx, min(T)), name = '0 s')
for i in range (nt+1):
  T = T = np.random.normal(loc = 50, scale =1, size = t.size)
  fig.add_scatter(x = np.arange(nx), y = T, name = f'{i} s')
fig.update_layout(height = 800, width = 1100)
fig.show()